In [3]:
import pandas as pd
pd.set_option('display.max_rows', 50)

salesdetail_montly = pd.read_csv('salesdetailmonthly_20240425_050244975.csv')

salesdetail_montly

C:\Users\wndows\AppData\Local\Temp\ipykernel_17252\1547539662.py:4: DtypeWarning: Columns (2,9,58,84,97,98,104,110,111,137,167,184,186) have mixed types. Specify dtype option on import or set low_memory=False.
  salesdetail_montly = pd.read_csv('salesdetailmonthly_20240425_050244975.csv')


,RegionName,StoreName,AlternateStoreCode,StoreGSTNumber,BillDate,BillTime,BillNumber,CustomerCode,CustomerName,CustomerGSTNumber,...,BillRemarks,AlternateCode2,PurchaseUnit,BulkDescription,ItemCost,CostOfSale,PackageInformation,ProviderTaxCode,TaxPercentage,ProductFullName
0,GUJARAT,MEDKART A STADIUM,2,24AAMCM5391B1ZI,01/04/2024,9:27AM,SC1,487202.0,MINESH PANDYA,NaN,...,NaN,NaN,1,NaN,0,0,TABLET,NaN,12,CLONIDIME 100MCG TAB 1X30
1,GUJARAT,MEDKART A STADIUM,2,24AAMCM5391B1ZI,01/04/2024,9:27AM,SC1,487202.0,MINESH PANDYA,NaN,...,NaN,NaN,1,NaN,0,0,TABLET,NaN,12,CILNIMERIT 20MG TAB 1X10
2,GUJARAT,MEDKART A STADIUM,2,24AAMCM5391B1ZI,01/04/2024,10:14AM,SC2,776.0,MEHTAB RANGWALA,NaN,...,NaN,NaN,1,NaN,0,0,CAPSULE,NaN,12,CALCIQUICK D31K CAP 1X10
3,GUJARAT,MEDKART A STADIUM,2,24AAMCM5391B1ZI,01/04/2024,10:14AM,SC2,776.0,MEHTAB RANGWALA,NaN,...,NaN,NaN,1,NaN,0,0,CAPSULE,NaN,12,CALCIQUICK D31K CAP 1X10
4,GUJARAT,MEDKART A STADIUM,2,24AAMCM5391B1ZI,01/04/2024,10:14AM,SC2,776.0,MEHTAB RANGWALA,NaN,...,NaN,NaN,1,NaN,0,0,TABLET,NaN,12,CETJOINT FORTE TAB 1X30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257890,GUJARAT,CURE PHARMACY,676,24AARFC9888M1ZC,24/04/2024,8:10PM,SR8,10500.0,AKSHAY DALAL,NaN,...,NaN,NaN,1,NaN,0,0,DROPS,NaN,12,SYSTANE ULTRA DROPS 10ML
257891,GUJARAT,J M AND K ENTERPRISES,682,24GONPP7240L1ZB,24/04/2024,10:42PM,SR6,779460.0,viral bharvad,NaN,...,NaN,NaN,1,NaN,0,0,TABLET,NaN,18,NICOTEX 4MG TAB 1X29
257892,GUJARAT,MEDKART AN GAMDIVAD,78,24AAMCM5937B1ZI,24/04/2024,11:59AM,SR6,915043.0,NITABEN SONI,NaN,...,NaN,NaN,1,NaN,0,0,DIAPER,NaN,12,DIAPER ADULT MEDIUM
257893,GUJARAT,MEDKART A SURELIYA ESTATE,80,24AAMCM5391B1ZI,24/04/2024,11:43AM,SR10,919027.0,bharat bhai,NaN,...,NaN,NaN,1,NaN,0,0,TABLET,NaN,12,TELSAR 40MG TAB 1X15


In [5]:
salesdetail_montly.columns

Index(['RegionName', 'StoreName', 'AlternateStoreCode', 'StoreGSTNumber',
       'BillDate', 'BillTime', 'BillNumber', 'CustomerCode', 'CustomerName',
       'CustomerGSTNumber',
       ...
       'BillRemarks', 'AlternateCode2', 'PurchaseUnit', 'BulkDescription',
       'ItemCost', 'CostOfSale', 'PackageInformation', 'ProviderTaxCode',
       'TaxPercentage', 'ProductFullName'],
      dtype='object', length=196)

In [6]:
salesdetail_montly.describe()

,CustomerCode,Quantity,FreeQty,FreeMRPValue,BaseValue,OtherTax,TNGSTAmount,CST,SGST,CGST,...,ProductImage,AlphaBatchID,OrderReference,AlternateCode2,PurchaseUnit,BulkDescription,ItemCost,CostOfSale,ProviderTaxCode,TaxPercentage
count,257804.000000,257895.000000,257895.0,257895.0,257895.000000,257895.0,257895.0,257895.0,257895.000000,257895.000000,...,0.0,0.0,0.0,0.0,257895.0,0.0,257895.0,257895.0,0.0,257895.000000
mean,421839.185695,2.160263,0.0,0.0,104.172366,0.0,0.0,0.0,6.019480,6.019480,...,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0.0,NaN,11.823692
std,294125.257804,5.892535,0.0,0.0,214.906541,0.0,0.0,0.0,12.038401,12.038401,...,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,2.571055
min,3.000000,-142.000000,0.0,0.0,-32825.895000,0.0,0.0,0.0,-1969.553600,-1969.553589,...,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0.0,NaN,0.000000
25%,159735.000000,1.000000,0.0,0.0,31.355932,0.0,0.0,0.0,1.767900,1.767900,...,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0.0,NaN,12.000000
50%,374680.000000,1.000000,0.0,0.0,62.500000,0.0,0.0,0.0,3.602400,3.602400,...,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0.0,NaN,12.000000
75%,679816.000000,3.000000,0.0,0.0,119.910710,0.0,0.0,0.0,7.149100,7.149100,...,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0.0,NaN,12.000000
max,919405.000000,720.000000,0.0,0.0,32825.895000,0.0,0.0,0.0,1969.553600,1969.553589,...,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0.0,NaN,18.000000


In [7]:
salesdetail_montly.dropna(axis=1, how='all')

,RegionName,StoreName,AlternateStoreCode,StoreGSTNumber,BillDate,BillTime,BillNumber,CustomerCode,CustomerName,CustomerGSTNumber,...,IsBudgetedDiscount,New customer Type,StoreAbbrevation,BillRemarks,PurchaseUnit,ItemCost,CostOfSale,PackageInformation,TaxPercentage,ProductFullName
0,GUJARAT,MEDKART A STADIUM,2,24AAMCM5391B1ZI,01/04/2024,9:27AM,SC1,487202.0,MINESH PANDYA,NaN,...,No,EXISTING,SA,NaN,1,0,0,TABLET,12,CLONIDIME 100MCG TAB 1X30
1,GUJARAT,MEDKART A STADIUM,2,24AAMCM5391B1ZI,01/04/2024,9:27AM,SC1,487202.0,MINESH PANDYA,NaN,...,No,EXISTING,SA,NaN,1,0,0,TABLET,12,CILNIMERIT 20MG TAB 1X10
2,GUJARAT,MEDKART A STADIUM,2,24AAMCM5391B1ZI,01/04/2024,10:14AM,SC2,776.0,MEHTAB RANGWALA,NaN,...,No,EXISTING,SA,NaN,1,0,0,CAPSULE,12,CALCIQUICK D31K CAP 1X10
3,GUJARAT,MEDKART A STADIUM,2,24AAMCM5391B1ZI,01/04/2024,10:14AM,SC2,776.0,MEHTAB RANGWALA,NaN,...,No,EXISTING,SA,NaN,1,0,0,CAPSULE,12,CALCIQUICK D31K CAP 1X10
4,GUJARAT,MEDKART A STADIUM,2,24AAMCM5391B1ZI,01/04/2024,10:14AM,SC2,776.0,MEHTAB RANGWALA,NaN,...,No,EXISTING,SA,NaN,1,0,0,TABLET,12,CETJOINT FORTE TAB 1X30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257890,GUJARAT,CURE PHARMACY,676,24AARFC9888M1ZC,24/04/2024,8:10PM,SR8,10500.0,AKSHAY DALAL,NaN,...,No,EXISTING,NaN,NaN,1,0,0,DROPS,12,SYSTANE ULTRA DROPS 10ML
257891,GUJARAT,J M AND K ENTERPRISES,682,24GONPP7240L1ZB,24/04/2024,10:42PM,SR6,779460.0,viral bharvad,NaN,...,No,EXISTING,NaN,NaN,1,0,0,TABLET,18,NICOTEX 4MG TAB 1X29
257892,GUJARAT,MEDKART AN GAMDIVAD,78,24AAMCM5937B1ZI,24/04/2024,11:59AM,SR6,915043.0,NITABEN SONI,NaN,...,No,EXISTING,NaN,NaN,1,0,0,DIAPER,12,DIAPER ADULT MEDIUM
257893,GUJARAT,MEDKART A SURELIYA ESTATE,80,24AAMCM5391B1ZI,24/04/2024,11:43AM,SR10,919027.0,bharat bhai,NaN,...,No,NEW,NaN,NaN,1,0,0,TABLET,12,TELSAR 40MG TAB 1X15


In [8]:
salesdetail_montly.describe()

,CustomerCode,Quantity,FreeQty,FreeMRPValue,BaseValue,OtherTax,TNGSTAmount,CST,SGST,CGST,...,ProductImage,AlphaBatchID,OrderReference,AlternateCode2,PurchaseUnit,BulkDescription,ItemCost,CostOfSale,ProviderTaxCode,TaxPercentage
count,257804.000000,257895.000000,257895.0,257895.0,257895.000000,257895.0,257895.0,257895.0,257895.000000,257895.000000,...,0.0,0.0,0.0,0.0,257895.0,0.0,257895.0,257895.0,0.0,257895.000000
mean,421839.185695,2.160263,0.0,0.0,104.172366,0.0,0.0,0.0,6.019480,6.019480,...,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0.0,NaN,11.823692
std,294125.257804,5.892535,0.0,0.0,214.906541,0.0,0.0,0.0,12.038401,12.038401,...,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,2.571055
min,3.000000,-142.000000,0.0,0.0,-32825.895000,0.0,0.0,0.0,-1969.553600,-1969.553589,...,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0.0,NaN,0.000000
25%,159735.000000,1.000000,0.0,0.0,31.355932,0.0,0.0,0.0,1.767900,1.767900,...,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0.0,NaN,12.000000
50%,374680.000000,1.000000,0.0,0.0,62.500000,0.0,0.0,0.0,3.602400,3.602400,...,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0.0,NaN,12.000000
75%,679816.000000,3.000000,0.0,0.0,119.910710,0.0,0.0,0.0,7.149100,7.149100,...,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0.0,NaN,12.000000
max,919405.000000,720.000000,0.0,0.0,32825.895000,0.0,0.0,0.0,1969.553600,1969.553589,...,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0.0,NaN,18.000000


In [ ]:
def sumQuantity(quan1, quan2):
    quan1 = quan1.split()
    quan2 = quan2.split()
    if len(quan1) > 1 and len(quan2) > 1:
        return int(quan1[0])
    else:
        return str(int())


In [57]:
display(pd.DataFrame({'Quantity': salesdetail_monthly['Quantity']}))

NameError: name 'salesdetail_monthly' is not defined

In [58]:
CustomerProductPurchase = {}
for group_name, group_data in salesdetail_montly.groupby(['RegionName', 'StoreName']):
    for group_customer, customer_data in group_data.groupby(['CustomerCode']):
        for billNumber, customer_purchases in customer_data.groupby('BillNumber'):
            CustomerProductPurchase[group_customer[0]] = []
            product_lister = []
            for productcode, customer_purchase in customer_purchases.groupby(['ProductCode']):
                product_lister.append(customer_purchase)
            # same product is packed more than one time
            if len(product_lister) > 1:
                for index in range(len(product_lister)-1,1):
                    product_lister[0]['Quantity'] = str(int(product[0]['Quantity']) + int(product[index]['Quantity']))
            CustomerProductPurchase[group_customer[0]].append(product_lister[0])

In [53]:
num = 210
for customer_code in CustomerProductPurchase:
    for product_code in CustomerProductPurchase[customer_code]:
        for product_bill in CustomerProductPurchase[customer_code][product_code]:
            display(pd.DataFrame({
                'RegionName': product_bill['RegionName'],
                'StoreName': product_bill['StoreName'],
                'CustomerCode': product_bill['CustomerCode'],
                'ProductFullName': product_bill['ProductFullName'],
                'ProductCode': product_bill['ProductCode'], 
                'Quantity': product_bill['Quantity'],
                'BillNumber': product_bill['BillNumber'],
                'BillDate': product_bill['BillDate'],
                'BillTime': product_bill['BillTime']
            }))
    if num == 0:
        break
    num -= 1

,RegionName,StoreName,CustomerCode,ProductFullName,ProductCode,Quantity,BillNumber,BillDate,BillTime
216285,GUJARAT,A-ONE MEDICOS,462.0,SYNDOPA 110MG TAB 1X15,19000,2,SC634,20/04/2024,10:36AM


,RegionName,StoreName,CustomerCode,ProductFullName,ProductCode,Quantity,BillNumber,BillDate,BillTime
254441,GUJARAT,A-ONE MEDICOS,2759.0,ORTHOTUS SPRAY 75GM,19409,1,SC779,24/04/2024,10:52PM


,RegionName,StoreName,CustomerCode,ProductFullName,ProductCode,Quantity,BillNumber,BillDate,BillTime
222968,GUJARAT,A-ONE MEDICOS,5768.0,UDP AT 5/50MG TAB 1X14,14081,3,SC681,21/04/2024,5:54PM


,RegionName,StoreName,CustomerCode,ProductFullName,ProductCode,Quantity,BillNumber,BillDate,BillTime
147095,GUJARAT,A-ONE MEDICOS,5871.0,COFBIT SYP 100ML,18522,1,SC448,13/04/2024,8:46PM


,RegionName,StoreName,CustomerCode,ProductFullName,ProductCode,Quantity,BillNumber,BillDate,BillTime
254452,GUJARAT,A-ONE MEDICOS,5978.0,ATORVASTATUS ASP GOLD 10MG TAB 1X15,17839,2,SC783,24/04/2024,11:19PM
254453,GUJARAT,A-ONE MEDICOS,5978.0,ATORVASTATUS ASP GOLD 10MG TAB 1X15,17839,4,SC783,24/04/2024,11:19PM


...

,RegionName,StoreName,CustomerCode,ProductFullName,ProductCode,Quantity,BillNumber,BillDate,BillTime
222962,GUJARAT,A-ONE MEDICOS,423040.0,CASISHIELD D SOLUTION 1LIT,2127,1,SC677,21/04/2024,5:44PM


,RegionName,StoreName,CustomerCode,ProductFullName,ProductCode,Quantity,BillNumber,BillDate,BillTime
185169,GUJARAT,A-ONE MEDICOS,423702.0,ESOTUS DSR CAP 1X10,16171,3,SC550,17/04/2024,8:00PM


,RegionName,StoreName,CustomerCode,ProductFullName,ProductCode,Quantity,BillNumber,BillDate,BillTime
124585,GUJARAT,A-ONE MEDICOS,425500.0,EMERTAM 0.4MG TAB 1X10,17838,1,SC362,11/04/2024,11:28AM


,RegionName,StoreName,CustomerCode,ProductFullName,ProductCode,Quantity,BillNumber,BillDate,BillTime
164194,GUJARAT,A-ONE MEDICOS,426335.0,ORTHOTUS GEL 75GM,18686,1,SC491,15/04/2024,6:36PM


,RegionName,StoreName,CustomerCode,ProductFullName,ProductCode,Quantity,BillNumber,BillDate,BillTime
47909,GUJARAT,A-ONE MEDICOS,426964.0,HUMAN ACTRAPID 40IU VIAL 10ML,6275,2,SC133,04/04/2024,12:27PM


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

# Sample data (replace this with your actual dataset)
data = {
    'time': pd.date_range('2022-01-01', periods=100),
    'quantity': np.random.randint(50, 200, 100),
    'sales': np.random.randint(1000, 5000, 100)
}
df = pd.DataFrame(data)

# Convert time to numeric values for regression (optional)
df['time_numeric'] = pd.to_numeric(df['time'])

# Perform linear regression
X = df[['time_numeric', 'quantity']]
y = df['sales']
model = LinearRegression().fit(X, y)

# Generate predicted sales values
df['predicted_sales'] = model.predict(X)

# Plotting
plt.figure(figsize=(10, 6))
plt.scatter(df['time'], df['sales'], label='Actual Sales')
plt.plot(df['time'], df['predicted_sales'], color='red', label='Predicted Sales')
plt.xlabel('Time')
plt.ylabel('Sales')
plt.title('Simple Linear Regression: Time vs. Sales')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [30]:
for customer_product_data in CustomerProductPurchase:
    

(462.0,)
(2759.0,)
(5768.0,)
(5871.0,)
(5978.0,)
(7278.0,)
(12366.0,)
(18022.0,)
(19447.0,)
(19892.0,)
(24469.0,)
(28305.0,)
(29637.0,)
(31368.0,)
(31394.0,)
(33210.0,)
(38038.0,)
(38601.0,)
(42923.0,)
(43675.0,)
(45577.0,)
(46908.0,)
(49255.0,)
(49436.0,)
(49510.0,)
(50684.0,)
(50747.0,)
(52387.0,)
(52838.0,)
(54385.0,)
(56932.0,)
(56951.0,)
(58380.0,)
(60258.0,)
(64696.0,)
(68146.0,)
(68866.0,)
(69088.0,)
(69252.0,)
(69259.0,)
(72063.0,)
(73031.0,)
(75095.0,)
(76287.0,)
(78847.0,)
(79847.0,)
(88586.0,)
(88729.0,)
(88995.0,)
(90209.0,)
(90946.0,)
(93523.0,)
(95006.0,)
(96257.0,)
(96324.0,)
(98278.0,)
(98946.0,)
(102967.0,)
(103037.0,)
(106419.0,)
(110284.0,)
(113055.0,)
(113773.0,)
(113943.0,)
(117578.0,)
(118743.0,)
(120783.0,)
(120835.0,)
(122150.0,)
(123582.0,)
(128394.0,)
(132371.0,)
(133224.0,)
(137015.0,)
(139484.0,)
(149513.0,)
(150393.0,)
(150826.0,)
(151230.0,)
(154123.0,)
(157310.0,)
(157366.0,)
(159758.0,)
(164297.0,)
(172262.0,)
(172324.0,)
(172849.0,)
(174050.0,)
(178815.

In [20]:
# how it all works lets understand
# different POS has different data so we gonna select that customer
# lets consider all 
# it's old thing dbscan, apriori, knn, kmeans, svm, decision tree
# feature analysis


# what her question was what you wanna understand, if she was data analysis, 
# firstly she would have known what there is to know
# 
# it also depends on Manufacturer type like forexample we are saleing one manufacturer,
# by taking frequency you are taking whole as a thing
# 
# one customer -> hasmany relation with different data type
# we consider as a week. on which day he will come


# we gonna make find pattern by making sliding windows
# we should be taking is frequency now 
# on top, price of product, 

# customer and it's relations
# 
# don't explain him
# why did i 


salesdetail_montly.describe()

,CustomerCode,Quantity,FreeQty,FreeMRPValue,BaseValue,OtherTax,TNGSTAmount,CST,SGST,CGST,...,ProductImage,AlphaBatchID,OrderReference,AlternateCode2,PurchaseUnit,BulkDescription,ItemCost,CostOfSale,ProviderTaxCode,TaxPercentage
count,257804.000000,257895.000000,257895.0,257895.0,257895.000000,257895.0,257895.0,257895.0,257895.000000,257895.000000,...,0.0,0.0,0.0,0.0,257895.0,0.0,257895.0,257895.0,0.0,257895.000000
mean,421839.185695,2.160263,0.0,0.0,104.172366,0.0,0.0,0.0,6.019480,6.019480,...,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0.0,NaN,11.823692
std,294125.257804,5.892535,0.0,0.0,214.906541,0.0,0.0,0.0,12.038401,12.038401,...,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,2.571055
min,3.000000,-142.000000,0.0,0.0,-32825.895000,0.0,0.0,0.0,-1969.553600,-1969.553589,...,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0.0,NaN,0.000000
25%,159735.000000,1.000000,0.0,0.0,31.355932,0.0,0.0,0.0,1.767900,1.767900,...,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0.0,NaN,12.000000
50%,374680.000000,1.000000,0.0,0.0,62.500000,0.0,0.0,0.0,3.602400,3.602400,...,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0.0,NaN,12.000000
75%,679816.000000,3.000000,0.0,0.0,119.910710,0.0,0.0,0.0,7.149100,7.149100,...,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0.0,NaN,12.000000
max,919405.000000,720.000000,0.0,0.0,32825.895000,0.0,0.0,0.0,1969.553600,1969.553589,...,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0.0,NaN,18.000000
